In [1]:
import sounddevice as sd
import librosa
import numpy as np
import joblib
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler



In [3]:

def extract_features(signal, sr):
    features = {}
    
    # Fundamental frequency features
    pitches = librosa.yin(signal, fmin=50, fmax=300, sr=sr)
    features['MDVP:Fo(Hz)'] = pitches.mean()
    features['MDVP:Fhi(Hz)'] = pitches.max()
    features['MDVP:Flo(Hz)'] = pitches.min()
    
    # Jitter features
    features['MDVP:Jitter(%)'] = np.sqrt(np.mean(signal ** 2))
    features['MDVP:Jitter(Abs)'] = np.mean(np.abs(signal))
    features['MDVP:RAP'] = np.sqrt(np.mean(signal ** 2))
    features['MDVP:PPQ'] = np.sqrt(np.mean(signal ** 2))
    features['Jitter:DDP'] = features['MDVP:RAP'] * 3
    
    # Shimmer features
    features['MDVP:Shimmer'] = np.sqrt(np.mean(signal ** 2))
    features['MDVP:Shimmer(dB)'] = np.sqrt(np.mean(signal ** 2))
    features['MDVP:APQ'] = np.sqrt(np.mean(signal ** 2))
    features['Shimmer:DDA'] = features['MDVP:APQ'] * 3
    features['shimmer:APQ3'] = np.sqrt(np.mean(signal ** 2))
    features['shimmer:APQ5'] = np.sqrt(np.mean(signal ** 2))

    # Other features
    features['NHR'] = np.sqrt(np.mean(signal ** 2))
    features['HNR'] = np.sqrt(np.mean(signal ** 2))
    features['RPDE'] = np.sqrt(np.mean(signal ** 2))
    features['DFA'] = np.sqrt(np.mean(signal ** 2))
    features['spread1'] = -np.sqrt(np.mean(signal ** 2))
    features['spread2'] = np.sqrt(np.mean(signal ** 2))
    features['D2'] = np.sqrt(np.mean(signal ** 2))
    features['PPE'] = np.sqrt(np.mean(signal ** 2))
    
    return features


In [4]:
# Set the sample rate for recording and processing the audio
sample_rate = 22050  # Sample rate in Hz
duration = 10  # Duration in seconds


In [5]:
# Record audio from the microphone
print('Recording audio...')
recording = sd.rec(int(duration * sample_rate), samplerate=sample_rate, channels=1)
sd.wait()  # Wait for the recording to complete


Recording audio...


In [6]:
# Flatten the recording array to one-dimensional
signal = recording.flatten()


In [7]:
# Extract features from the recorded audio
extracted_features = extract_features(signal, sample_rate)


In [8]:
# Print the extracted features
for feature, value in extracted_features.items():
    print(f'{feature}: {value}')
    

MDVP:Fo(Hz): 153.1582271517756
MDVP:Fhi(Hz): 302.05479452054794
MDVP:Flo(Hz): 50.0
MDVP:Jitter(%): 0.06002994626760483
MDVP:Jitter(Abs): 0.037192828953266144
MDVP:RAP: 0.06002994626760483
MDVP:PPQ: 0.06002994626760483
Jitter:DDP: 0.18008983880281448
MDVP:Shimmer: 0.06002994626760483
MDVP:Shimmer(dB): 0.06002994626760483
MDVP:APQ: 0.06002994626760483
Shimmer:DDA: 0.18008983880281448
shimmer:APQ3: 0.06002994626760483
shimmer:APQ5: 0.06002994626760483
NHR: 0.06002994626760483
HNR: 0.06002994626760483
RPDE: 0.06002994626760483
DFA: 0.06002994626760483
spread1: -0.06002994626760483
spread2: 0.06002994626760483
D2: 0.06002994626760483
PPE: 0.06002994626760483


In [9]:
# Round off the extracted features up to 5 decimal places
rounded_features = {}
for key, value in extracted_features.items():
    rounded_features[key] = round(value, 5)
    print(rounded_features[key])

153.15823
302.05479
50.0
0.06003
0.03719
0.06003
0.06003
0.18009
0.06003
0.06003
0.06003
0.18009
0.06003
0.06003
0.06003
0.06003
0.06003
0.06003
-0.06003
0.06003
0.06003
0.06003


In [10]:
# Preprocess the extracted features

feature_vector = []
feature_order = ['MDVP:Fo(Hz)', 'MDVP:Fhi(Hz)', 'MDVP:Flo(Hz)', 'MDVP:Jitter(%)', 'MDVP:Jitter(Abs)', 'MDVP:RAP',
                 'MDVP:PPQ', 'Jitter:DDP', 'MDVP:Shimmer', 'MDVP:Shimmer(dB)', 'MDVP:APQ', 'Shimmer:DDA','shimmer:APQ3','shimmer:APQ5',
                 'NHR', 'HNR', 'RPDE', 'DFA', 'spread1', 'spread2', 'D2', 'PPE']

for feature in feature_order:
    feature_vector.append(rounded_features[feature])

print(feature_vector)

[153.15823, 302.05479, 50.0, 0.06003, 0.03719, 0.06003, 0.06003, 0.18009, 0.06003, 0.06003, 0.06003, 0.18009, 0.06003, 0.06003, 0.06003, 0.06003, 0.06003, 0.06003, -0.06003, 0.06003, 0.06003, 0.06003]


In [ ]:
scaler = StandardScaler()
scaler.fit()

In [15]:
scaler = StandardScaler()
test_data=[['91.90400', '115.87100', '86.29200', '0.00540', '0.00006', '0.00281', '0.00336', '0.00844', '0.02752', '0.24900', '0.02214', '0.04272', '0.01424', '0.01641', '0.01141', '21.41400', '0.583390', '0.792520', '-4.960234', '0.363566', '2.642476', '0.275931'],
['136.92600', '159.86600', '131.27600', '0.00293', '0.00002', '0.00118', '0.00153', '0.00355', '0.01259', '0.11200', '0.01140', '0.01968', '0.00656', '0.00717', '0.00581', '25.70300', '0.460600', '0.646846', '-6.547148', '0.152813', '2.041277', '0.138512'],
['139.17300', '179.13900', '76.55600', '0.00390', '0.00003', '0.00165', '0.00208', '0.00496', '0.01642', '0.15400', '0.01797', '0.02184', '0.00728', '0.00932', '0.01041', '24.88900', '0.430166', '0.665833', '-5.660217', '0.254989', '2.519422', '0.199889'],
['152.84500', '163.30500', '75.83600', '0.00294', '0.00002', '0.00121', '0.00149', '0.00364', '0.01828', '0.15800', '0.01246', '0.03191', '0.01064', '0.00972', '0.00609', '24.92200', '0.474791', '0.654027', '-6.105098', '0.203653', '2.125618', '0.170100'],
['209.51600', '253.01700', '89.48800', '0.00564', '0.00003', '0.00331', '0.00292', '0.00994', '0.02751', '0.26300', '0.01879', '0.04812', '0.01604', '0.01657', '0.01810', '19.14700', '0.431674', '0.683244', '-6.195325', '0.129303', '2.784312', '0.168895'],
['174.68800', '240.00500', '74.28700', '0.01360', '0.00008', '0.00624', '0.00564', '0.01873', '0.02308', '0.25600', '0.01667', '0.03804', '0.01268', '0.01365', '0.10715', '17.88300', '0.407567', '0.655683', '-6.787197', '0.158453', '2.679772', '0.131728'],
['198.76400', '396.96100', '74.90400', '0.00740', '0.00004', '0.00370', '0.00390', '0.01109', '0.02296', '0.24100', '0.01588', '0.03794', '0.01265', '0.01321', '0.07223', '19.02000', '0.451221', '0.643956', '-6.744577', '0.207454', '2.138608', '0.123306'],
['214.28900', '260.27700', '77.97300', '0.00567', '0.00003', '0.00295', '0.00317', '0.00885', '0.01884', '0.19000', '0.01373', '0.03078', '0.01026', '0.01161', '0.04398', '21.20900', '0.462803', '0.664357', '-5.724056', '0.190667', '2.555477', '0.148569']]
new_data =[]
new_data += feature_vector
new_data += test_data
print(new_data)

[153.15823, 302.05479, 50.0, 0.06003, 0.03719, 0.06003, 0.06003, 0.18009, 0.06003, 0.06003, 0.06003, 0.18009, 0.06003, 0.06003, 0.06003, 0.06003, 0.06003, 0.06003, -0.06003, 0.06003, 0.06003, 0.06003, ['91.90400', '115.87100', '86.29200', '0.00540', '0.00006', '0.00281', '0.00336', '0.00844', '0.02752', '0.24900', '0.02214', '0.04272', '0.01424', '0.01641', '0.01141', '21.41400', '0.583390', '0.792520', '-4.960234', '0.363566', '2.642476', '0.275931'], ['136.92600', '159.86600', '131.27600', '0.00293', '0.00002', '0.00118', '0.00153', '0.00355', '0.01259', '0.11200', '0.01140', '0.01968', '0.00656', '0.00717', '0.00581', '25.70300', '0.460600', '0.646846', '-6.547148', '0.152813', '2.041277', '0.138512'], ['139.17300', '179.13900', '76.55600', '0.00390', '0.00003', '0.00165', '0.00208', '0.00496', '0.01642', '0.15400', '0.01797', '0.02184', '0.00728', '0.00932', '0.01041', '24.88900', '0.430166', '0.665833', '-5.660217', '0.254989', '2.519422', '0.199889'], ['152.84500', '163.30500', '

In [22]:
input_data_as_numpy_array = [np.array(arr) for arr in new_data]

# reshape the numpy array
input_data_reshaped = input_data_as_numpy_array.reshape(1,-1)

# standardize the data

new_data_scaled = scaler.transform(input_data_reshaped)
print(new_data_scaled)


AttributeError: 'list' object has no attribute 'reshape'

In [24]:
# Convert the feature vector to a numpy array
feature_vector = np.array(feature_vector).reshape(1, -1)

# Round the feature vector to 5 decimal places precision
rounded_feature_vector = np.round(feature_vector, decimals=5)
# Modify display settings to disable scientific notation
np.set_printoptions(precision=5, suppress=True)

print(rounded_feature_vector)

#print(feature_vector)

[[153.15823 302.05479  50.        0.06003   0.03719   0.06003   0.06003
    0.18009   0.06003   0.06003   0.06003   0.18009   0.06003   0.06003
    0.06003   0.06003   0.06003   0.06003  -0.06003   0.06003   0.06003
    0.06003]]


In [26]:
scaler=StandardScaler()
new_data_scaled=scaler.transform(rounded_feature_vector)
print(new_data_scaled)

NotFittedError: This StandardScaler instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.

In [99]:
# Load the pre-trained SVM model
svm_model = joblib.load(D:\User\Documents\codes\Parkinsons-disease-detection\pythonNoteBooks\SVM_model.pkl')

In [100]:
# Perform prediction using the SVM model
prediction = svm_model.predict(feature_vector)


In [104]:
# Print the prediction
print(f"Prediction: {prediction}")

Prediction: [0]
